In [108]:
import vk
import pandas as pd
import json
import time
from pandas import json_normalize

In [109]:

v='5.89'
api = vk.API(access_token='vk1.a._KsTsa8wau2jsv2iCt80iIj-z2gu7Y978RoBwh4jnZOZymVH_nR4EFJ-7Jj0tqL1SAA6T9jnVOXKMLlGUL1O3-gH8KmBirDjBe7AbrzfBsd5voGelouCQTWcasNqpRpxUKjEIftzpbahGTPyIi5Coxu2lR_rtjIMMwgyyPpq3S_JlTpnKFcNTf5ClHu1hHySNtmEEEVTnWtVjhVsJCVCpQ',v=v)




In [110]:
users=['https://vk.com/yourdevils18','https://vk.com/privet.andreii','https://vk.com/danbase']
for i,user in enumerate(users):
    users[i]=user.replace('https://vk.com/','')


def getUserId(users):
    users_ids=[]
    for user in users:
        user_id=api.users.get(user_id=user)[0]['id']
        users_ids.append(user_id)
    return users_ids


users_ids=getUserId(users)
users_ids



[436090772, 371599152, 259775630]

In [117]:
df=pd.DataFrame()
df_main=pd.DataFrame()

def getGroups(user_id):
    data_gr=api.groups.get(user_id=user_id,count=1000)
    print(data_gr)
    df[['Groups_Q','Groups_count']] = pd.json_normalize(data_gr)



def getFriends(user_id):
    data_fr=api.friends.get(user_id=user_id,count=1000)
    print(data_fr)
    df[['Friends_Q','friends_count']] = pd.json_normalize(data_fr)

def getFollowers(user_id):
    data_fol=api.users.getFollowers(user_id=user_id,count=1000)
    df[['User_Folowers','user_f_ids']] = pd.json_normalize(data_fol)

def getWall(owner_id):
    data_wall=api.wall.get(owner_id=owner_id,count=1000)
    df[['Wall_Q','Wall_count']]= pd.json_normalize(data_wall)


    data_wall_owner=api.wall.get(owner_id=owner_id,count=1000,filter='owner')
    df[['Wall_Q_owner','Wall_count_owner']]= pd.json_normalize(data_wall_owner)

    data_wall_owner=api.wall.get(owner_id=owner_id,count=1000,filter='others')
    df[['Wall_Q_others','Wall_count_others']]= pd.json_normalize(data_wall_owner)


def getPhotos(owner_id):
    data_photos_com=api.photos.getAllComments(owner_id=owner_id,count=100)['count']
    df[['Photos_C']]= data_photos_com

    comments_strings = []
    comments = api.photos.getAllComments(owner_id=owner_id,count=100)['items']
    comments_strings.append([comment['text'] for comment in comments])
    df['comments_photo']=comments_strings

def getStatus(owner_id):
    data_status=api.status.get(owner_id=owner_id)
    df[['Status_txt']]= pd.json_normalize(data_status)

def getAvatar(user_id):
    data_Avatar=api.users.get(user_ids=user_id,fields='has_photo')[0]['has_photo']
    print(data_Avatar)
    df['Avatar']=data_Avatar

#Надо получить все что в fields(Пока только about получается)
def getUserInfo(owner_id):
    print(owner_id)
    data_info=api.users.get(user_ids=owner_id,fields=['about','activities','interests','sex','movies','music','screen_name'])[0]['about']
    df[['about','activities','interests','sex','movies','music','screen_name']]=data_info




for user_id in users_ids:
    print(user_id)
    time.sleep(3)
    getFriends(user_id)
    getFollowers(user_id)
    getGroups(user_id)
    time.sleep(3)
    getWall(user_id)
    getPhotos(user_id)
    getStatus(user_id)
    time.sleep(3)
    getAvatar(user_id)
    getUserInfo(user_id)
    df['user_id']=str(user_id)
    df
    df_main=pd.concat([df_main,df],ignore_index=True)
df_main.head()


436090772
{'count': 68, 'items': [9309774, 11097036, 14527278, 31696367, 50942452, 54963874, 65288289, 69238545, 79026232, 83746278, 110944777, 112635468, 128021100, 136236702, 138681848, 155850934, 161898956, 162185289, 168410616, 171552430, 175004783, 176960226, 186051060, 204187414, 206343941, 206810939, 209358996, 213381410, 217792834, 222097213, 223708328, 227385047, 228094579, 232196719, 235138208, 239082935, 242453578, 245987815, 254903449, 259142245, 259775630, 259997686, 260436059, 264897645, 267272297, 274256334, 313160962, 320128151, 320395898, 323654475, 332890817, 334226357, 338236043, 343753700, 353273135, 362624128, 371599152, 374127310, 387665230, 394027876, 394070022, 417076950, 460374034, 493286957, 497702571, 510031375, 717068122, 718399676]}
{'count': 100, 'items': [187289777, 73408049, 153296449, 178282306, 153823581, 134506306, 146062039, 29725575, 215633780, 211200114, 75561291, 197808636, 191623389, 213209830, 78522286, 64729046, 200063444, 58265344, 167138278, 

TypeError: unhashable type: 'list'